# Prompt
The programming problem that we would like you to take on is to write a program that identifies the areas of light in sample TIFF images from a digital microscope.

To give you some context. Our microscopes acquire images as bitmaps. Each bitmap has a number light spots that look like circular smudges. These are generated by the light emitted from the florescent molecules that are within the sample, hitting the detector. The first part of super resolution microscopy is to locate these smudges.

The input to your code should be a TIFF image. The images have some noise (this is normal). You can assume that each smudge is 6x6 pixels. You should be able to run your code and output a list of rectangles that bound the smudges that you have detected. Your code should be resilient to the noise and any artifacts that appear in the image and find only the circular smudges. The programming problem should take you no more than two or three evenings. Your code should be clean, efficient, and of a quality that you feel would be suitable to be production code. There are 3rd party libraries, system utilities etc available that you can use to read TIFF images so don’t write your own TIFF reader ;-)

Looking at the images will give you an intuitive idea of the noise that you will encounter in the image. The background of the image is not uniform but appears "mottled", occasionally there are single stray pixels that are very bright. Additionally the smudges themselves are not at all uniform but are themselves subject to noise (you will see that they are somewhat mottled). If you consider the frame as a whole, athough the light smudges which we are identifying are very visible (easy to spot with the naked eye) statistically they contribute very little to the overall brightness of the image.

For this challenge we are not concerned about user interface (a command line tool is fine), or how you read the images you can use whatever library or frameworks that you want - however we would like you to write the code that processes the bitmap image and discovers the positions of the smudges yourself.

If you have any questions just drop us an email.

We supply a few of test images that we would like your program to work with.

Finally we would like you to submit clean, simple, well-structured code that you would consider ready for submission to a professional project.

In [ ]:
# import the necessary packages
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Constants
blur_size = (9, 9)
padding = 5 # padding for rectangle

# Visualize Plot Variables
showOrignalPlot = False
showPlotsWithRectangles = False

# Export Variables
exportImagesWithRectangles = False # Exports original image with rectangles as tif
exportSmudgeLocations = True # Exports to CSV

In [ ]:
# import images
path = 'images/'
fileType = '.tif'
images = [(cv2.imread(file), file) for file in glob.glob(path + "*" + fileType)]

In [ ]:
for i in range(len(images)):
    output = images[i][0]
    # Grayscale image to help distinguish important visual features
    gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    
    # Blurring image to reduce noise
    blurred = cv2.GaussianBlur(gray, blur_size, 0)
    
    # Threshold the image to reveal light regions in the blurred image
    thresh = cv2.adaptiveThreshold(blurred,
                                   255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY,
                                   35,
                                   0
                                  )
    if showOrignalPlot:
        plt.imshow(thresh)
        plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
        plt.title(images[i][1][len(path):-len(fileType)])
        plt.show()
        
    # Apply Hough transform on the image
    circles = cv2.HoughCircles(thresh,
                               cv2.HOUGH_GRADIENT,
                               1,
                               minDist = 10,
                               param1 = 200,
                               param2 = 3,
                               minRadius = 3,
                               maxRadius = 9
                               )
    
    # Draw circles if any were detected by Hough transform
    if circles is not None:
        circles = np.uint16(np.around(circles[0, :]))
        for (x, y, r) in circles:
            # Draw rectangle
            cv2.rectangle(output,
                          (x - (r + padding), y - (r + padding)),
                          (x + (r + padding), y + (r + padding)),
                          (int(r), 128 + int(x), 255 - int(y)),
                          3
                         )
        if showPlotsWithRectangles:
            plt.imshow(output)
            plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
            plt.title(images[i][1][len(path):-len(fileType)] + '__detected_{}_smudges'.format(circles.shape[0]))
            plt.show()
        if exportImagesWithRectangles:
            cv2.imwrite(images[i][1][len(path):-len(fileType)] + '__detected_{}_smudges'.format(circles.shape[0]) + fileType,
                            output)
        if exportSmudgeLocations:
            np.savetxt(images[i][1][len(path):-len(fileType)] + '__detected_{}_smudges'.format(circles.shape[0]) + ".csv",
                       circles,
                       delimiter = ","
                      )
print('Complete')